<div dir="rtl" style="text-align: center;">

# لیگ علم داده
### انجمن هوش مصنوعی دانشگاه خوارزمی - کارگروه کاوش عملی
#### هفته سوم: مدلسازی و ارزیابی 
**نام و نام خانوادگی:**  
**سرگروه:**  
**تاریخ ارسال:** 

---

</div>

<div dir="rtl">

### چالش 1: 

فرض کنید شما مدیر تیم دیتا هستید. دو کارآموز جدید برای پروژه استخدام کرده‌اید و هر کدام یک مدل برای پیش‌بینی نمرات ارائه داده‌اند

* **کارآموز الف (Parsa):** مدلی با خطای RMSE = 1.07 آورده است.
* **کارآموز ب (Sara):** مدلی با خطای خیره‌کننده RMSE = 0.22 آورده و ادعا می‌کند مدلش تقریباً بدون نقص است!

مدیر پروژه هیجان‌زده است و می‌خواهد همین امروز مدل سارا (کد دوم) را روی سرور دانشگاه مستقر کند چون خطای آن نزدیک به صفر است. اما قبل از استقرار تصمیم گرفته تا با شما مشورت کند.

الف) تحلیل خود را از کد های سارا و پارسا بنویسید، هر کدام از چه روش ها و مدل هایی استفاده کردند، مزایا و معایت هر کدام چیست؟

ب) یک گزارش فنی کوتاه برای "رد" یا "تایید" این مدل‌ها بنویسید.
 و توصیه نهایی خود را به مدیر پروژه اعلام کنید

</div>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.decomposition import PCA


train_data = pd.read_csv("train.csv")


categorical_cols = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'schoolsup', 
                    'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 
                    'Mjob', 'Fjob', 'reason', 'guardian']
ordinal_cols = ['Medu', 'Fedu', 'age', 'traveltime', 'studytime', 'failures', 
                'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health']
numeric_cols = ['G1', 'G2', 'G3', 'absences']


train_data['Fedu_Medu'] = train_data['Fedu'] + train_data['Medu']
train_data['Walc_Dalc'] = train_data['Walc'] + train_data['Dalc']
train_data['G1_G2'] = train_data['G1'] + train_data['G2']

ordinal_cols = [col for col in ordinal_cols if col not in ['Fedu', 'Medu', 'Walc', 'Dalc']]
numeric_cols = [col for col in numeric_cols if col not in ['G1', 'G2']]
numeric_cols += ['Fedu_Medu', 'Walc_Dalc', 'G1_G2']

X = train_data.drop('G3', axis=1)
y = train_data['G3']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_copy = X_train.copy()
X_val_copy = X_val.copy()

X_train_copy['G3'] = y_train
X_val_copy['G3'] = y_val

def elliptic_envelope_outliers(df, cols):
    ee = EllipticEnvelope(contamination=0.05, random_state=42)
    df['anomaly'] = ee.fit_predict(df[cols])
    df = df[df['anomaly'] == 1].drop('anomaly', axis=1)
    return df


X_train_ee = elliptic_envelope_outliers(X_train_copy.copy(), numeric_cols)
X_val_ee = elliptic_envelope_outliers(X_val_copy.copy(), numeric_cols)

y_train_ee = X_train_ee.pop('G3')
y_val_ee = X_val_ee.pop('G3')


categorical_cols = [col for col in categorical_cols if col in X_train_ee.columns]
ordinal_cols = [col for col in ordinal_cols if col in X_train_ee.columns]
numeric_cols = [col for col in numeric_cols if col in X_train_ee.columns]


def build_and_evaluate_model(preprocessor, X_train, X_val, y_train, y_val):
    pca = PCA(n_components=0.95)
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('pca', pca),
        ('model', LinearRegression())
    ])
    
 
    pipeline.fit(X_train, y_train)
    

    y_pred = pipeline.predict(X_val)
    
 
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return rmse


preprocessor_robust = ColumnTransformer(
    transformers=[
        ('ordinal', OrdinalEncoder(), categorical_cols + ordinal_cols),  
        ('scaler', RobustScaler(), numeric_cols)
    ],
    remainder='passthrough'
)


rmse_robust_pca = build_and_evaluate_model(preprocessor_robust, X_train_ee, X_val_ee, y_train_ee, y_val_ee)
print(f"RMSE after Robust Scaling and PCA with Label Encoding: {rmse_robust_pca}")


RMSE after Robust Scaling and PCA with Label Encoding: 1.0732279270607483


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import StackingRegressor


train_data = pd.read_csv("train.csv")


categorical_cols = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'schoolsup', 
                    'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 
                    'Mjob', 'Fjob', 'reason', 'guardian']
ordinal_cols = ['Medu', 'Fedu', 'age', 'traveltime', 'studytime', 'failures', 
                'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health']
numeric_cols = ['G1', 'G2', 'G3', 'absences']


train_data['Fedu_Medu'] = train_data['Fedu'] + train_data['Medu']
train_data['Walc_Dalc'] = train_data['Walc'] + train_data['Dalc']
train_data['G1_G2'] = train_data['G1'] + train_data['G2']


ordinal_cols = [col for col in ordinal_cols if col not in ['Fedu', 'Medu', 'Walc', 'Dalc']]
numeric_cols = [col for col in numeric_cols if col not in ['G1', 'G2']]
numeric_cols += ['Fedu_Medu', 'Walc_Dalc', 'G1_G2']


encoder = OrdinalEncoder()
train_data[categorical_cols + ordinal_cols] = encoder.fit_transform(train_data[categorical_cols + ordinal_cols])


scaler = StandardScaler()
train_data[numeric_cols] = scaler.fit_transform(train_data[numeric_cols])

X = train_data.drop('G3', axis=1)
y = train_data['G3']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_copy = X_train.copy()
X_val_copy = X_val.copy()

X_train_copy['G3'] = y_train
X_val_copy['G3'] = y_val


from sklearn.covariance import EllipticEnvelope
def elliptic_envelope_outliers(df, cols):
    ee = EllipticEnvelope(contamination=0.05, random_state=42)
    df['anomaly'] = ee.fit_predict(df[cols])
    df = df[df['anomaly'] == 1].drop('anomaly', axis=1)
    return df


X_train_ee = elliptic_envelope_outliers(X_train_copy.copy(), numeric_cols)
X_val_ee = elliptic_envelope_outliers(X_val_copy.copy(), numeric_cols)

y_train_ee = X_train_ee.pop('G3')
y_val_ee = X_val_ee.pop('G3')


models = {
    'RandomForest': RandomForestRegressor(),
    'ExtraTrees': ExtraTreesRegressor(),
    'CatBoost': CatBoostRegressor(silent=True),
    'GradientBoosting': GradientBoostingRegressor()
}


def kfold_cross_validation_evaluation(models, X, y):
    results = {}
    
    # KFold با 5 بخش
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for model_name, model in models.items():
        # پایپ‌لاین برای هر مدل
        pipeline = Pipeline(steps=[
            ('model', model)
        ])
        
        rmse_list = []
        for train_index, val_index in kf.split(X):
            X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
            y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
            
            pipeline.fit(X_train_fold, y_train_fold)
            y_pred = pipeline.predict(X_val_fold)
            
            rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
            rmse_list.append(rmse)
        
        avg_rmse = np.mean(rmse_list)
        results[model_name] = avg_rmse
        print(f'{model_name} K-Fold Cross-Validated RMSE: {avg_rmse}')
    
    return results

kfold_cv_results = kfold_cross_validation_evaluation(models, X_train_ee, y_train_ee)

print(kfold_cv_results)


stacking_model = StackingRegressor(
    estimators=[
        ('RandomForest', RandomForestRegressor()),
        ('ExtraTrees', ExtraTreesRegressor()),
        ('CatBoost', CatBoostRegressor(silent=True)),
        ('GradientBoosting', GradientBoostingRegressor())
    ],
    final_estimator=GradientBoostingRegressor() 
)


def evaluate_stacking_model(model, X, y):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_list = []
    
    for train_index, val_index in kf.split(X):
        X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
        y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
        
        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_val_fold)
        
        rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
        rmse_list.append(rmse)
    
    avg_rmse = np.mean(rmse_list)
    print(f'Stacking Model K-Fold Cross-Validated RMSE: {avg_rmse}')
    return avg_rmse


stacking_rmse = evaluate_stacking_model(stacking_model, X_train_ee, y_train_ee)


RandomForest K-Fold Cross-Validated RMSE: 0.221450047966295
ExtraTrees K-Fold Cross-Validated RMSE: 0.2526895846633275
CatBoost K-Fold Cross-Validated RMSE: 0.24739834632359012
GradientBoosting K-Fold Cross-Validated RMSE: 0.23681217246769765
{'RandomForest': 0.221450047966295, 'ExtraTrees': 0.2526895846633275, 'CatBoost': 0.24739834632359012, 'GradientBoosting': 0.23681217246769765}
Stacking Model K-Fold Cross-Validated RMSE: 0.2770533467873001


<div dir="rtl" style="width: 100%; overflow: hidden; white-space: nowrap; text-align: center; margin: 60px 0; user-select: none;">
    <span style="color: #d97706; font-size: 18px; letter-spacing: 15px; opacity: 0.6;">
        ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆
    </span>
    <div style="font-size: 12px; color: #9ca3af; margin-top: -10px; background: #fff; display: inline-block; padding: 0 15px; position: relative; bottom: 12px;">
        Next Challenge
    </div>
</div>

<div dir="rtl">

### چالش 2: برقراری عدالت در مدل


پس از اینکه توابع پاکسازی و مهندسی ویژگی که جلسه پیش ساختید را اجرا کردید و با توجه به نکاتی در چالش قبل یاد گرفتید خودتان یک پایپ‌لاین کامل شامل مدل دلخواهتان بنویسید (رگرسیون یا درختی یا ...)؛ ابتدا `RMSE` را روی کل ولیدیشن حساب کنید. سپس `RMSE` را به تفکیک روی مدارس `GP` و `MS` جداگانه بررسی کنید. آیا مدل شما در هر دو مدرسه به یک اندازه خوب عمل می‌کند؟

---

#### ماموریت ۱: اصلاح استراتژی تقسیم داده (Stratified Splitting)

حالا که مشکل را دیدید، باید آن را حل کنید. یکی از دلایل اصلی این مشکل، نحوه تقسیم داده‌ها (`train_test_split`) است. اگر تصادفی تقسیم کنید، ممکن است در داده‌های `Train` کثرت نمونه‌های `MS` آنقدر کم باشد که مدل اصلاً الگوی آن‌ها را یاد نگیرد.

* **اقدام:** کد تقسیم داده خود را اصلاح کنید. به جای تقسیم رندوم ساده، از روش **"Stratified Splitting"** استفاده کنید تا مطمئن شوید نسبت مدرسه `MS` در داده‌های `Train` و `Test` حفظ می‌شود.
* **راهنمایی:** در تابع `train_test_split` پارامتری به نام `stratify` وجود دارد. آن را بر اساس ستون `school` تنظیم کنید.
* ** حالا دوباره مدل را آموزش دهید و `RMSE`های تفکیک شده را چک کنید. آیا فاصله بین خطای دو مدرسه کمتر شد؟

---

#### ماموریت ۲: تنظیم وزن نمونه‌ها (Sample Weighting)

اگر `Stratification` کافی نبود، باید زورِ مدل را زیاد کنید! به مدل بگویید: "اشتباه کردن روی دانش‌آموزان مدرسه `MS` هزینه بیشتری دارد!"

* **اقدام:** وزن هر نمونه را محاسبه کنید (**Compute Sample Weights**). به دانش‌آموزان `MS` وزن بیشتری بدهید (معکوس فراوانی‌شان). مثلاً اگر نسبت `GP` به `MS` برابر ۹ به ۱ است، وزن `MS` را ۹ برابر `GP` کنید. این وزن‌ها را هنگام آموزش به مدل بدهید (اکثر مدل‌های Scikit-Learn مثل `RandomForest` در متد `.fit()` پارامتری به نام `sample_weight` دارند).
* **نتیجه نهایی:** آیا با این کار، عدالت برقرار شد؟ (ممکن است  کل کمی زیاد شود، اما  مدرسه `MS` باید کاهش یابد).

---

#### بخش امتیازی: بررسی ناتوازنی در هدف (Target Imbalance)

شما در این سوال با مفهوم **Feature Imbalance** آشنا شدید، اما یک ناتوازنی دیگر هم می‌تواند وجود داشته باشد که بررسی نکردیم: **Target Imbalance**.

* این بار `RMSE` به تفکیک نمره باشد؛ مثلاً سه دسته دانش‌آموز ضعیف، متوسط و قوی.
* آیا وزن سه دسته یکسان است؟
* اگر یکسان نیست چه راهکاری پیشنهاد می‌دهید؟

</div>


<div dir="rtl" style="width: 100%; overflow: hidden; white-space: nowrap; text-align: center; margin: 60px 0; user-select: none;">
    <span style="color: #d97706; font-size: 18px; letter-spacing: 15px; opacity: 0.6;">
        ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆
    </span>
    <div style="font-size: 12px; color: #9ca3af; margin-top: -10px; background: #fff; display: inline-block; padding: 0 15px; position: relative; bottom: 12px;">
        Next Challenge
    </div>
</div>

<div dir="rtl">

### چالش 3: ترکیب و مقایسه مدل ها

<div dir="rtl">

پس از اینکه پایپ‌لاین‌های پاک‌سازی و مهندسی ویژگی هفته قبل را اجرا کردید، وقت انتخاب "مدل" است. ما سه کاندیدا محبوب داریم:

* **Linear Regression:** سریع، ساده، اما شاید ضعیف در برابر الگوهای پیچیده.
* **Lasso Regression:** هوشمند در حذف ویژگی‌های اضافه (Feature Selection خودکار).
* **Random Forest:** قدرتمند و غیرخطی، اما با خطر Overfitting.

---

#### ماموریت ۱: ارزیابی اولیه و اعتبارسنجی متقاطع

از `cross_val_score` با `cv=5` استفاده کنید تا میانگین خطای ($RMSE$) هر سه مدل را بدست آورید.

* **تحلیل نموداری:** یک نمودار **Boxplot** بکشید که توزیع خطاهای این ۵ فولد را برای هر سه مدل نشان دهد.
* **سوال:** چرا در این پروژه، شبکه عصبی را جزو گزینه‌های محبوب نیاوردیم؟



---

#### ماموریت ۲: تشخیص بیش‌برازش (Overfitting) و پایداری

یک تابع بنویسید که برای هر سه مدل کارهای زیر را انجام دهد:

1.  مدل را روی کل `X_train` فیت کند و $RMSE$ روی خودِ `X_train` را حساب کند (نام آن را بگذارید: **Train Error**).
2.  با استفاده از `cross_val_score` (با `cv=5`)، میانگین $RMSE$ روی داده‌های اعتبارسنجی را حساب کند (نام آن را بگذارید: **CV Error**).

* **تحلیل نموداری (Bar Plot):** یک نمودار میله‌ای گروهی (**Grouped Bar Plot**) رسم کنید. برای هر مدل، دو میله کنار هم داشته باشید: یکی برای Train Error و یکی برای CV Error.



**سوالات تحلیلی:**
* در کدام مدل، ارتفاع دو میله تقریباً برابر است؟ (نشانه پایداری/Underfitting).
* در کدام مدل، میله‌ی Train بسیار کوتاه اما میله‌ی CV بلند است؟ (نشانه Overfitting).
* بر اساس اصل **"تیغ اوکام" (Occam's Razor)**، اگر خطای CV دو مدل نزدیک به هم بود، کدام را انتخاب می‌کنید؟ مدلی که شکاف کمتری دارد یا مدلی که پیچیده‌تر است؟

---

#### ماموریت ۳: تشکیل شورا (Weighted Averaging)

حالا که نقاط قوت و ضعف هر مدل را دیدید، بیایید به جای انتخاب یک نفر، یک "شورا" تشکیل دهیم.

**اقدام:**
سه سناریوی مختلف برای وزن‌دهی (**Weighted Averaging**) تعریف کنید و $RMSE$ نهایی را روی داده‌های Validation محاسبه کنید:
1.  **سناریوی دموکراسی (Simple Average):** به همه مدل‌ها وزن برابر بدهید. 
2.  **سناریوی نخبه‌گرا (Best Model Dominance):** به مدلی که در ماموریت ۱ کمترین خطای CV را داشت، وزن ۰.۷ و به بقیه ۰.۱۵ بدهید.
3.  **سناریوی محافظه‌کار (Conservative / Occam’s Style):** به مدل‌های خطی (Lasso/Linear) مجموعاً وزن ۰.۸ و به Random Forest وزن ۰.۲ بدهید.

**تحلیل نهایی:**
* کدام ترکیب برنده شد؟
* اگر ترکیب محافظه‌کار برنده شد، دلیلش چیست؟ (آیا RF نویزها را یاد گرفته بود که با کم کردن وزنش، دقت بالا رفت؟).
* اگر ترکیب نخبه‌گرا برنده شد، آیا ریسک Overfitting در داده‌های جدید (Test) وجود دارد؟

</div>

<div dir="rtl">

#### راهنمایی برای ماموریت ۲

برای رسم نمودارهای این بخش، می‌توانید از ساختار قطعه کد زیر الگوبرداری کنید:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def compare_models_visual(models, X, y):
    model_names = []
    train_errors = []
    cv_errors = []
    
    for name, model in models.items():
        model.fit(X, y)
        y_pred_train = model.predict(X)
        train_rmse = np.sqrt(mean_squared_error(y, y_pred_train))
        
        cv_scores = -cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
        cv_rmse = cv_scores.mean()
        
        model_names.append(name)
        train_errors.append(train_rmse)
        cv_errors.append(cv_rmse)
    
    x = np.arange(len(model_names))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, train_errors, width, label='Train Error (Overfitting Check)', color='skyblue')
    rects2 = ax.bar(x + width/2, cv_errors, width, label='CV Error (Real Performance)', color='salmon')
    
    ax.set_ylabel('RMSE')
    ax.set_title('Train vs Validation Error: Detecting Overfitting')
    ax.set_xticks(x)
    ax.set_xticklabels(model_names)
    ax.legend()
    
    plt.show()

# اجرا
# models = {'Linear': LinearRegression(), 'Lasso': Lasso(alpha=0.1), 'RandomForest': RandomForestRegressor()}
# compare_models_visual(models, X_train, y_train)

<div dir="rtl" style="width: 100%; overflow: hidden; white-space: nowrap; text-align: center; margin: 60px 0; user-select: none;">
    <span style="color: #d97706; font-size: 18px; letter-spacing: 15px; opacity: 0.6;">
        ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆
    </span>
    <div style="font-size: 12px; color: #9ca3af; margin-top: -10px; background: #fff; display: inline-block; padding: 0 15px; position: relative; bottom: 12px;">
        Next Challenge
    </div>
</div>

<div dir="rtl">

### چالش 4: (Hyperparameter Tuning)

مدل Random Forest پتانسیل بالایی دارد، اما تنظیمات پیش‌فرض آن (default parameters) بهینه نیستند. ما می‌خواهیم با تنظیم دقیق پیچ‌ومهره‌های مدل، خطا را کاهش دهیم.

#### ماموریت ۱
یک `GridSearchCV` طراحی کنید که پارامترهای زیر را بررسی کند:

* **n_estimators:** تعداد درخت‌ها (مثلاً ۵۰، ۱۰۰، ۲۰۰)
* **max_depth:** عمق هر درخت (برای جلوگیری از حفظ کردن داده‌ها - مثلاً ۱۰، ۲۰، `None`)
* **min_samples_split:** حداقل نمونه برای انشعاب.

**گزارش:**
بهترین پارامترها چه بودند؟ آیا محدود کردن `max_depth` باعث کاهش Overfitting شد؟



---

#### ماموریت ۲
۱. از داخل مدلِ بهینه شده‌ی خود (Best Estimator)، اولین درخت را استخراج کنید (در sklearn با دستور `model.estimators_[0]` قابل دسترسی است).
۲. با استفاده از تابع `plot_tree`، نمودار این درخت را رسم کنید.
۳. مقایسه تصویری:
* یک بار نمودار را برای مدلی با `max_depth=None` (آزاد) بکشید.
* یک بار نمودار را برای مدلی با `max_depth=3` (محدود) بکشید.



**سوال تحلیلی:**
به شاخ و برگ‌های درخت اول (عمق نامحدود) نگاه کنید. آیا گره‌هایی را می‌بینید که فقط ۱ نمونه در آن‌ها وجود دارد؟

</div>

<div dir="rtl">

#### راهنمایی برای ماموریت ۲

برای رسم نمودار این بخش، می‌توانید از ساختار قطعه کد زیر الگوبرداری کنید:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

def visualize_tree(rf_model, feature_names, max_depth=None):
    # انتخاب اولین درخت از جنگل
    single_tree = rf_model.estimators_[0]
    
    plt.figure(figsize=(20, 10))
    plot_tree(single_tree, 
              feature_names=feature_names, 
              filled=True, 
              rounded=True, 
              max_depth=max_depth, # فقط تا عمق مشخصی را نشان بده تا خوانا باشد
              fontsize=10)
    plt.title(f"Decision Tree Visualization (Max Depth: {max_depth})")
    plt.show()

# نحوه استفاده:
# visualize_tree(best_rf_model, X_train.columns, max_depth=3)

<div dir="rtl" style="width: 100%; overflow: hidden; white-space: nowrap; text-align: center; margin: 60px 0; user-select: none;">
    <span style="color: #d97706; font-size: 18px; letter-spacing: 15px; opacity: 0.6;">
        ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆
    </span>
    <div style="font-size: 12px; color: #9ca3af; margin-top: -10px; background: #fff; display: inline-block; padding: 0 15px; position: relative; bottom: 12px;">
        Next Challenge
    </div>
</div>

<div dir="rtl">

### چالش 5: پیش‌بینی بازه اطمینان (Quantile Regression)

تا الان همه مدل‌ها تلاش می‌کردند بگویند: "نمره علی ۱۴ می‌شود". اما در دنیای واقعی، ما باید ریسک را هم بسنجیم. مدیر مدرسه می‌خواهد بداند: "بدترین حالت و بهترین حالت نمره علی چند است؟"
مدل‌های معمولی (MSE Loss) میانگین را پیش‌بینی می‌کنند. ما می‌خواهیم چارک‌ها (Quantiles) را پیش‌بینی کنیم.

####  ماموریت:
به جای یک مدل، ۳ مدل `GradientBoostingRegressor` آموزش دهید:

1. با `loss='quantile'` و `alpha=0.05` (پیش‌بینی بدبینانه - کف نمره).
2. با `loss='quantile'` و `alpha=0.50` (همان میانه - پیش‌بینی نرمال).
3. با `loss='quantile'` و `alpha=0.95` (پیش‌بینی خوش‌بینانه - سقف نمره).

#### خروجی:
برای ۵ دانش‌آموز از داده تست، نموداری بکشید که یک خط عمودی (بازه اطمینان) داشته باشد.



**مثال:** مدل می‌گوید نمره سارا با اطمینان ۹۰٪ بین ۱۲ تا ۱۷ خواهد بود.

**سوال تحلیلی:** دانش‌آموزی که بازه اطمینانش کوچک است (مثلاً ۱۵ تا ۱۶) با دانش‌آموزی که بازه بزرگی دارد (۱۰ تا ۱۸) چه تفاوتی در ویژگی‌ها دارند؟ (کشف عدم قطعیت).

</div>

<div dir="rtl" style="width: 100%; overflow: hidden; white-space: nowrap; text-align: center; margin: 60px 0; user-select: none;">
    <span style="color: #d97706; font-size: 18px; letter-spacing: 15px; opacity: 0.6;">
        ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆
    </span>
    <div style="font-size: 12px; color: #9ca3af; margin-top: -10px; background: #fff; display: inline-block; padding: 0 15px; position: relative; bottom: 12px;">
        Next Challenge
    </div>
</div>

<div dir="rtl">

### چالش 6: تحلیل متقابل (Counterfactual Analysis)

مدیر مدرسه نمی‌خواهد فقط نمره را بداند، او می‌خواهد بداند "چطور نمره را تغییر دهد؟". در علم داده سنتی، ما فقط همبستگی (Feature Importance) را می‌بینیم. اما در اینجا می‌خواهیم تأثیر تغییر یک ویژگی را شبیه‌سازی کنیم. این مفهوم به نام **Causal Inference** یا استنتاج علیتی نزدیک است.

####  ماموریت:
یک دانش‌آموز ضعیف (نمره زیر ۱۰) را از داده‌های تست انتخاب کنید.

1.  بدون تغییر دادن بقیه ویژگی‌ها، فقط مقدار `studytime` او را یکی‌یکی زیاد کنید (۱، ۲، ۳، ۴).
2.  بدون تغییر دادن بقیه، فقط مقدار `absences` او را صفر کنید.
3.  نمره جدید را با مدل پیش‌بینی کنید.



#### نمودار "What-If":
نموداری بکشید که محور افقی آن "تغییرات" باشد و محور عمودی "نمره پیش‌بینی شده".

**سوال :** برای این دانش‌آموز خاص، کدام استراتژی موثرتر است؟ "افزایش ۱ ساعت مطالعه" یا "کاهش ۱۰ جلسه غیبت"؟
(این چالش نشان می‌دهد که نسخه پیچیده شده برای هر دانش‌آموز متفاوت است؛ برای یکی مطالعه مهم است، برای دیگری حضور در کلاس).

</div>

<div dir="rtl" style="width: 100%; overflow: hidden; white-space: nowrap; text-align: center; margin: 60px 0; user-select: none;">
    <span style="color: #d97706; font-size: 18px; letter-spacing: 15px; opacity: 0.6;">
        ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆ ◆
    </span>
    <div style="font-size: 12px; color: #9ca3af; margin-top: -10px; background: #fff; display: inline-block; padding: 0 15px; position: relative; bottom: 12px;">
        Next Challenge
    </div>
</div>

<div dir="rtl">

### تمرین نهایی (پیش‌بینی روی داده‌های واقعی)

تا الان هر مدلی تست می‌کردید داشتید روی مجموعه `validation` تست می‌کردید. حالا می‌خواهیم شما کد نهایی خود را آماده کنید شامل پایپ‌لاین‌های پاک‌سازی، مهندسی ویژگی و مدل‌سازی و در نهایت یک فایل پیش‌بینی برای داده‌های تست که ستون `G3` آن خالی است انجام دهید.

بنابراین شما در نهایت باید یک فایل `CSV` به سرگروه‌های خود تحویل دهید که شامل پیش‌بینی ستون هدف باشد. نتایج شما با داده‌های نتایج واقعی تطبیق داده می‌شود و با معیار $RMSE$ نتیجه مدل نهایی شما مشخص می‌شود.

---

### راهنمای ارسال فایل نهایی (Submission Guide)

برای اینکه مدل شما در سیستم داوری خودکار نمره بگیرد، فایل ارسالی باید دقیقاً طبق استاندارد زیر باشد. هرگونه مغایرت باعث می‌شود سیستم نمره شما را محاسبه نکند.

#### ۱. فرمت فایل
* فایل باید فرمت `csv` باشد.
* نام فایل ترجیحاً `submission.csv` باشد.

#### ۲. ساختار محتوا
* فایل باید فقط یک ستون داشته باشد.
* نام هدر (**Header**) این ستون باید دقیقاً **G3** باشد.
* تعداد ردیف‌ها باید دقیقاً برابر با تعداد ردیف‌های فایل `test.csv` باشد.
* **بسیار مهم:** ترتیب ردیف‌ها نباید تغییر کند. پیش‌بینی ردیف اول فایل شما، باید مربوط به دانش‌آموز ردیف اول فایل `test.csv` باشد.

#### ۳. کد ذخیره‌سازی استاندارد
برای اطمینان از صحت فایل، لطفاً در انتهای نوت‌بوک خود، از قطعه کد زیر برای ساخت فایل خروجی استفاده کنید:


In [ ]:
# فرض کنید y_pred آرایه پیش‌بینی‌های نهایی شماست (روی داده‌های تست)

import pandas as pd

# ساخت دیتافریم
submission = pd.DataFrame(y_pred, columns=['G3'])

# ذخیره بدون ایندکس (بسیار مهم: index=False)
submission.to_csv('submission.csv', index=False)

print("✅ فایل submission.csv با موفقیت ساخته شد.")

<div dir="rtl">

### ❌ اشتباهات رایج (که باعث رد شدن مدل می‌شود):

* **ذخیره با ایندکس:** اگر `index=False` را نگذارید، فایل شما دو ستون خواهد داشت (شماره ردیف و نمره). سیستم داوری ممکن است شماره ردیف را به اشتباه به عنوان نمره بخواند و خطای شما بسیار زیاد شود!
* **بهم ریختن ترتیب:** اگر داده‌های تست را `Shuffle` کرده باشید، پیش‌بینی‌ها جابجا می‌شوند.
* **تغییر نام ستون:** نام ستون نباید `Grade` یا `Pred` باشد، فقط **G3**.

</div>